In [14]:
import sys
sys.path.append("/data/Beatriz/Doctorado GR/ADL_platform//S-ADL")


In [15]:
from SADL.time_series.time_series_datasets_uci import global_load as load_time_series 
from sklearn.model_selection import train_test_split
X,y = load_time_series('ai4i_2020_predictive_maintenance_dataset')   #name dataset in static datasets uci repo
labels = y["Machine failure"]
X = X.drop('Type',axis=1)  # remove Type or apply one hot encoding

Metadata: {'uci_id': 601, 'name': 'AI4I 2020 Predictive Maintenance Dataset', 'repository_url': 'https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/601/data.csv', 'abstract': 'The AI4I 2020 Predictive Maintenance Dataset is a synthetic dataset that reflects real predictive maintenance data encountered in industry.', 'area': 'Computer Science', 'tasks': ['Classification', 'Regression', 'Causal-Discovery'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 10000, 'num_features': 6, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], 'index_col': ['UID', 'Product ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Wed Feb 14 2024', 'dataset_doi': '10.24432/C5HS5C', 'creators': [], 'intro_paper': {'ID': 386, 'type': 'NATIVE', 'title': 'Explainable Artificial 

In [16]:
from SADL.time_series.preprocessing.preprocessing_ts import StandardScalerPreprocessing 
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )


(8000, 5) (2000, 5) (8000,) (2000,)


In [17]:
from SADL.time_series.algorithms import transformers
from SADL.time_series.time_series_utils import TimeSeriesProcessor
import numpy as np
import torch

batch_size = 16
seq_len = 24
input_dim = 5  # tanto encoder como decoder
d_model = 64

# Simulamos series de entrada
X = torch.randn(batch_size, seq_len, input_dim)

# Instanciamos el modelo Transformer
kwargs ={
    "algorithm_": "Transformer",
    "label_parser": lambda scores: (scores > np.percentile(scores, 95)).astype(int),
    "size_enc_in":input_dim,
    "size_dec_in": input_dim,
    "ulayers_feedfwd": 128,
    "seq_len":seq_len,
    "d_qk":64,
    "d_v":64,
    "d_model":d_model,
    "n_layers": 2,
    "n_heads": 8,
   "dropout_rate": 0.1,
   "attns_outs":False, 
    "train_epochs": 10,
    "batch_size": batch_size,
    "lr": 1e-3
    }
    

model = transformers.TransformersAnomalyDetection(**kwargs)

Train Params: {'label_parser': <function <lambda> at 0x7a68bb329e10>, 'train_epochs': 10, 'batch_size': 16, 'lr': 0.001} 
Model Params: {'algorithm_': 'Transformer', 'size_enc_in': 5, 'size_dec_in': 5, 'ulayers_feedfwd': 128, 'seq_len': 24, 'd_qk': 64, 'd_v': 64, 'd_model': 64, 'n_layers': 2, 'n_heads': 8, 'dropout_rate': 0.1, 'attns_outs': False}


In [12]:
processor = TimeSeriesProcessor(window_size= seq_len, step_size=1, future_prediction=False)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

(8000, 5) (2000, 5) (8000,) (2000,)


In [ ]:

model.fit(X_train)


In [4]:
scores = model.decision_function(X)
labels = model.predict(X)

print("Anomaly scores:", scores.shape)
print("Predicted labels:", labels.shape)


Anomaly scores: torch.Size([16])
Predicted labels: torch.Size([384, 5])
